In [1]:
#### import modules

# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

import numpy as np
import arcpy
import os
import sys
from arcpy.sa import *
import pandas as pd
import shutil


# Some functions to solve no data values in asc files

# function to check ASC files for no data values
def Check_no_data_asc(yourFileName):
    data = np.loadtxt(yourFileName, skiprows=6)              # read the grid part of the  .asc file 
    if -9999 in np.unique(data): 
        print("There are {} no data values in {}".format(np.count_nonzero(data == -9999), yourFileName))    


# Function to clean .asc files of no data values

def Clean_no_data_ESRI_trash_asc(yourFileName, Replace_values_with):

    data = np.loadtxt(yourFileName, skiprows=6)              # read the grid part of the  .asc file 
    print("Starting unique values are: \n {}".format(np.unique(data)))                                   # see if -9999 is in there
    #np.shape(data)
    data[data == -9999] = Replace_values_with  

    header=[]                                  # Get the header rows as a big string
    fh = open(yourFileName,'r')
    for i,line in enumerate(fh):
        if i is 6: break
        header.append("{}".format(line))
    fh.close()
    header = ''.join(header)    

    # Save data as file...
    np.savetxt(yourFileName, data, fmt='%1i') 

    #We read the existing text from file in READ mode
    src=open(yourFileName,"r")
    fline=header    #Prepending string
    oline=src.readlines()
    #Here, we prepend the string we want to on first line
    oline.insert(0,fline)
    src.close()

    #We again open the file in WRITE mode 
    src=open(yourFileName,"w")
    src.writelines(oline)
    src.close()

    #double check
    data = np.loadtxt(yourFileName, skiprows=6)              # read the grid part of the  .asc file 
    print("Ending unique values are: \n {}".format(np.unique(data)))  

In [2]:
# set properties

cel_size = 50     # in m 

arcpy.env.overwriteOutput = True # make sure overwrite files is on
# projection definition 
sr_project = arcpy.SpatialReference(32702)   # Project dataset into WGS84


#### General coverages and paths. More, basic model setup.
GIS_FOLDER = os.path.join('..', '..', 'Raw_GIS_Data')
STD_INPUT_FOLDER = os.path.join('..', '..', 'Std_input')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')


# set/create Inpit file workspace
Generalworkspace = os.path.join('..', 'input/General')
if not os.path.exists(Generalworkspace):
    os.makedirs(Generalworkspace)
    
input_folder = os.path.join("..", "input") 
    
# this makes a domain grid of zeros of with cell size
Grid_rast0 =os.path.join(Generalworkspace, 'Grid_rast0')
arcpy.PolygonToRaster_conversion(os.path.join(GIS_FOLDER, 'grid_bound.shp'), "zero", Grid_rast0, cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )

# this makes a domain grid of ones of with cell size
Grid_rast1 =os.path.join(Generalworkspace, 'Grid_rast1')
arcpy.PolygonToRaster_conversion(os.path.join(GIS_FOLDER, 'grid_bound.shp'), "one", Grid_rast1, cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )

<Result '..\\input\\General\\Grid_rast1'>

### Shuttle in non-dynamic text tables

In [3]:
# Simple RO : RF ratios file
shutil.copy2(os.path.join(GIS_FOLDER, 'Runofftorainfall2\\RO_Rf_ratios_real_monthly3_2000_2010.txt') ,os.path.join(input_folder)) # note this is from the simplified version with zone IDs starting at 1

# Rain Fragments file
shutil.copy2(os.path.join(STD_INPUT_FOLDER, "Fragments", 'Rainfall_fragments_2001.prn') ,os.path.join(input_folder))  

#  Fragments sequence file
shutil.copy2(os.path.join(STD_INPUT_FOLDER, "Fragments", 'Sequence_file_2002.prn') ,os.path.join(input_folder)) 

'..\\input\\Sequence_file_2002.prn'

### ORIGINAL Land use Soils Data processing

In [4]:
### Land use data: incluing canopy coverage and % impervious surfaces 
# set/create workspace
workspace = os.path.join('..', 'input/Land_use_Soils_data')
if not os.path.exists(workspace):
    os.makedirs(workspace)

# project file using the simplified 3 m version LU_Big_simp_3
arcpy.Project_management(os.path.join(GIS_FOLDER, "Land_use\\LU_Big_simp_3.shp"), os.path.join(workspace, 'LU_shp_projected.shp'), sr_project)

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,os.path.join(workspace, 'LU_shp_projected.shp'),  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), os.path.join(workspace, 'LU_shp_projected.shp')], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "LU_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "LU_grid.asc"))

# canopy cover %ages with landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "fracCanCov", os.path.join(workspace, "CanCovRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "CanCovRas"), os.path.join(workspace, "CanCovRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "CanCovRas.asc"))

# % impervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Impv", os.path.join(workspace, "pImpvRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pImpvRas"), os.path.join(workspace, "pImpvRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pImpvRas.asc"))

# % pervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Perv", os.path.join(workspace, "pPERVRas"), cell_assignment="MAXIMUM_COMBINED_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pPERVRas"), os.path.join(workspace, "pPERVRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pPERVRas.asc"))


# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_raster'))
arcpy.Delete_management(os.path.join(workspace, 'CanCovRas'))
arcpy.Delete_management(os.path.join(workspace, 'pImpvRas'))
arcpy.Delete_management(os.path.join(workspace, 'pPERVRas'))



### Same with soil shapefile into WGS84 and then resterize to make soil moisture and h2o group rasters
# set/create workspace
workspace = os.path.join('..', 'input/Land_use_Soils_data')
if not os.path.exists(workspace):
    os.makedirs(workspace)

# Project dataset into WGS84
arcpy.Project_management(os.path.join(GIS_FOLDER, 'Soils\\Tut_Soil_clip2.shp'), os.path.join(workspace, 'soils_shp_projected.shp'), sr_project) 

arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'soils_shp_projected.shp'),  os.path.join(workspace, 'soils_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'soils_shp_bound.shp'), os.path.join(workspace, 'soils_shp_projected.shp')], os.path.join(workspace, 'soils_shp_ready.shp'))

# convert to raster
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'soils_shp_ready.shp'),  "H2O_grp", os.path.join(workspace, "H2Ogp_rst"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'soils_shp_ready.shp'),  "SM_INpFT", os.path.join(workspace, "SMC_rst"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)

arcpy.RasterToASCII_conversion(os.path.join(workspace, "SMC_rst"), os.path.join(workspace, "soil_moist_cap_grid.asc"))
arcpy.RasterToASCII_conversion(os.path.join(workspace, "H2Ogp_rst"), os.path.join(workspace, "H2Ogp_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "soil_moist_cap_grid.asc"))
Check_no_data_asc(os.path.join(workspace, "H2Ogp_grid.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'soils_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'soils_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'soils_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'SMC_rst'))
arcpy.Delete_management(os.path.join(workspace, 'H2Ogp_rst'))


# at 25 m cell size need to clean up pPERVRas.asc   using 30 as a random value that is represented a lot
Clean_no_data_ESRI_trash_asc(os.path.join(workspace, "pPERVRas.asc"), 30)

Starting unique values are: 
 [   0.   30.  100.]
Ending unique values are: 
 [   0.   30.  100.]


### Scen 1  Land use  Data processing

In [5]:
workspace = os.path.join('..', 'input/Land_use_Soils_data_Scen1')
if not os.path.exists(workspace):
    os.makedirs(workspace)

########   Main change Parameter HERE   ############    
LU_Scn_data_Path = os.path.join("..", "..", "Scripts_Scen1", "WorkSpaces", "Finished_LU_99.shp")
    

# project file using the SCENARIO VERSION
arcpy.Project_management(LU_Scn_data_Path, os.path.join(workspace, 'LU_shp_projected.shp'), sr_project)

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,os.path.join(workspace, 'LU_shp_projected.shp'),  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), os.path.join(workspace, 'LU_shp_projected.shp')], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "LU_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "LU_grid.asc"))

# canopy cover %ages with landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "fracCanCov", os.path.join(workspace, "CanCovRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "CanCovRas"), os.path.join(workspace, "CanCovRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "CanCovRas.asc"))

# % impervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Impv", os.path.join(workspace, "pImpvRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pImpvRas"), os.path.join(workspace, "pImpvRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pImpvRas.asc"))

# % pervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Perv", os.path.join(workspace, "pPERVRas"), cell_assignment="MAXIMUM_COMBINED_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pPERVRas"), os.path.join(workspace, "pPERVRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pPERVRas.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_raster'))
arcpy.Delete_management(os.path.join(workspace, 'CanCovRas'))
arcpy.Delete_management(os.path.join(workspace, 'pImpvRas'))
arcpy.Delete_management(os.path.join(workspace, 'pPERVRas'))

<Result 'true'>

### Scen 2  Land use  Data processing

In [6]:
workspace = os.path.join('..', 'input/Land_use_Soils_data_Scen2')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
########   Main change Parameter HERE   ############
LU_Scn_data_Path = os.path.join("..", "..", "Scripts_Scen2", "WorkSpaces", "Finished_LU_99.shp")
    

# project file using the SCENARIO VERSION
arcpy.Project_management(LU_Scn_data_Path, os.path.join(workspace, 'LU_shp_projected.shp'), sr_project)

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,os.path.join(workspace, 'LU_shp_projected.shp'),  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), os.path.join(workspace, 'LU_shp_projected.shp')], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "LU_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "LU_grid.asc"))

# canopy cover %ages with landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "fracCanCov", os.path.join(workspace, "CanCovRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "CanCovRas"), os.path.join(workspace, "CanCovRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "CanCovRas.asc"))

# % impervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Impv", os.path.join(workspace, "pImpvRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pImpvRas"), os.path.join(workspace, "pImpvRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pImpvRas.asc"))

# % pervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Perv", os.path.join(workspace, "pPERVRas"), cell_assignment="MAXIMUM_COMBINED_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pPERVRas"), os.path.join(workspace, "pPERVRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pPERVRas.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_raster'))
arcpy.Delete_management(os.path.join(workspace, 'CanCovRas'))
arcpy.Delete_management(os.path.join(workspace, 'pImpvRas'))
arcpy.Delete_management(os.path.join(workspace, 'pPERVRas'))

<Result 'true'>

### Scen 3  Land use  Data processing

In [7]:
workspace = os.path.join('..', 'input/Land_use_Soils_data_Scen3')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
########   Main change Parameter HERE   ############    
LU_Scn_data_Path = os.path.join("..", "..", "Scripts_Scen3", "WorkSpaces", "Finished_LU_99.shp")
    

# project file using the SCENARIO VERSION
arcpy.Project_management(LU_Scn_data_Path, os.path.join(workspace, 'LU_shp_projected.shp'), sr_project)

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,os.path.join(workspace, 'LU_shp_projected.shp'),  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), os.path.join(workspace, 'LU_shp_projected.shp')], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "LU_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "LU_grid.asc"))

# canopy cover %ages with landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "fracCanCov", os.path.join(workspace, "CanCovRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "CanCovRas"), os.path.join(workspace, "CanCovRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "CanCovRas.asc"))

# % impervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Impv", os.path.join(workspace, "pImpvRas"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pImpvRas"), os.path.join(workspace, "pImpvRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pImpvRas.asc"))

# % pervious raster from landuse map
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "pct_Perv", os.path.join(workspace, "pPERVRas"), cell_assignment="MAXIMUM_COMBINED_AREA",  cellsize=cel_size )
arcpy.RasterToASCII_conversion(os.path.join(workspace, "pPERVRas"), os.path.join(workspace, "pPERVRas.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "pPERVRas.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'LU_raster'))
arcpy.Delete_management(os.path.join(workspace, 'CanCovRas'))
arcpy.Delete_management(os.path.join(workspace, 'pImpvRas'))
arcpy.Delete_management(os.path.join(workspace, 'pPERVRas'))

<Result 'true'>

### Rainfall station zones

In [8]:
### Rasterize the rainfall station zones
workspace = os.path.join('..', 'input/Rain_stations')
if not os.path.exists(workspace):
    os.makedirs(workspace)

# project file 
arcpy.Project_management(os.path.join(GIS_FOLDER, 'Stations\\Thissen_poly_rain_clip_modified2.shp'), os.path.join(workspace, 'Thissen_poly_projected.shp'), sr_project)

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,os.path.join(workspace, 'Thissen_poly_projected.shp'),  os.path.join(workspace, 'Thissen_poly_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'Thissen_poly_bound.shp'), os.path.join(workspace, 'Thissen_poly_projected.shp')], os.path.join(workspace, 'Thissen_poly_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'Thissen_poly_ready.shp'), "Gage_ID", os.path.join(workspace, "TP_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "TP_raster"), os.path.join(workspace, "TP_grid.asc"))
# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "TP_grid.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'Thissen_poly_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Thissen_poly_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Thissen_poly_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'TP_raster'))

<Result 'true'>

### ORIGINAL  water lines  and OSDS location infiltration

In [9]:
### Rasterize the water lines to calculate direct infiltration from leaking lines
workspace = os.path.join('..', 'input/Direct_infiltration')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
# Calculate in the amount of NRW water equally distributed over the area of the water lines 
# note total ASPA 5 year production average = 12,740,000 and the Non-Rev water ave = 7,940,000 Gal/day   (264.172 gal per m3 of water) 
# conversions 5 year production average in m3/day = 48226.1557 and the Non-Rev water ave = 30056.175522 m3/day    
    
# create appropriately sized buffers of 1/2 the cell size
arcpy.Buffer_analysis(os.path.join(GIS_FOLDER, 'Direct_infiltration\\Transmission_water_mains.shp'), os.path.join(workspace, 'Water_lines_Buffer.shp'), "{} meters".format(cel_size/2), "FULL", "ROUND", "ALL")
# calculate area of water line polygons in m2
arcpy.AddField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "DOUBLE")    # add Active cell unit field
arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "AREA")    # calculate the area of the buffer zone
Area_list= [row[0] for row in arcpy.da.SearchCursor(os.path.join(workspace, 'Water_lines_Buffer.shp'), "POLY_AREA")]  # make a list of the area number
arcpy.CalculateField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "!Id! + 30056.1755/Area_list[0]*39.3701", "PYTHON3") # calculate the appropriate amount of infitration in inches to equal 7.94 MGD as spread over the whole water line influence area

    
# Project dataset into WGS84
arcpy.Project_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'), sr_project) 
arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'),  os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp')], os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))

arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'), "Infl_inch", os.path.join(workspace, "A_WL_Rast"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "A_WL_Rast"), os.path.join(workspace, "A_WL_Rast.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'A_WL_Rast'))



### Rasterize OSDS location infiltration
# FROM Shuler et al 2017   OSDS flow = 1.454203 m3/unit/Day
    
arcpy.env.extent = os.path.join('..', 'input/General/Grid_rast1')  # set processing extent to our desired model boundary
    
pdensOut = arcpy.sa.PointDensity(os.path.join(GIS_FOLDER, 'Direct_infiltration\\OSDS_units_pts.shp'),"NONE", cel_size, NbrCircle(1, "CELL"), "SQUARE_METERS" )
pdensOut.save(os.path.join(workspace, 'temp_dens'))   # calculate the OSDS density per m2

arcpy.Times_3d(os.path.join(workspace, 'temp_dens'), cel_size**2, os.path.join(workspace, 'OSDS_dens'))  # make the OSDS density in OSDS units, meaning number of units per cell

infiltration_in_inches_per_day_per_OSDS = (1.454203/cel_size**2)*39.3701

arcpy.Times_3d(os.path.join(workspace, 'OSDS_dens'), infiltration_in_inches_per_day_per_OSDS, os.path.join(workspace, 'OSDS_inlf_in'))  # calculate amount of water infiltrated in each cell in inches

arcpy.RasterToASCII_conversion(os.path.join(workspace, 'OSDS_inlf_in'), os.path.join(workspace, "OSDS_inlf_in.asc"))

arcpy.Delete_management(os.path.join(workspace, 'temp_dens'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_dens'))   # clean up workspace

#### add all the direct infiltration into one raster because apparently the control file wont take multiples???
arcpy.Plus_3d(os.path.join(workspace,  "A_WL_Rast.asc"), os.path.join(workspace, "OSDS_inlf_in.asc"), os.path.join(workspace, "WLOSDrast"))
arcpy.RasterToASCII_conversion(os.path.join(workspace, "WLOSDrast"), os.path.join(workspace, "Total_inlf_in.asc"))

# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "Total_inlf_in.asc"))

### Scen 1 water lines  and OSDS location infiltration

In [10]:
workspace = os.path.join('..', 'input/Direct_infiltration_Scen_1')
if not os.path.exists(workspace):
    os.makedirs(workspace)

########   Main change Parameter HERE   ############
DIFF_from_scenario_amount = .96

    
# create appropriately sized buffers of 1/2 the cell size
arcpy.Buffer_analysis(os.path.join(GIS_FOLDER, 'Direct_infiltration\\Transmission_water_mains.shp'), os.path.join(workspace, 'Water_lines_Buffer.shp'), "{} meters".format(cel_size/2), "FULL", "ROUND", "ALL")
# calculate area of water line polygons in m2
arcpy.AddField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "DOUBLE")    # add Active cell unit field
arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "AREA")    # calculate the area of the buffer zone
Area_list= [row[0] for row in arcpy.da.SearchCursor(os.path.join(workspace, 'Water_lines_Buffer.shp'), "POLY_AREA")]  # make a list of the area number
arcpy.CalculateField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "!Id! + 30056.1755/Area_list[0]*39.3701", "PYTHON3") # calculate the appropriate amount of infitration in inches to equal 7.94 MGD as spread over the whole water line influence area
    
# Project dataset into WGS84
arcpy.Project_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'), sr_project) 
arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'),  os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp')], os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))

arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'), "Infl_inch", os.path.join(workspace, "A_WL_Rast"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "A_WL_Rast"), os.path.join(workspace, "A_WL_Rast.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'A_WL_Rast'))



### Rasterize OSDS location infiltration
# FROM Shuler et al 2017   OSDS flow = 1.454203 m3/unit/Day

    
arcpy.env.extent = os.path.join('..', 'input/General/Grid_rast1')  # set processing extent to our desired model boundary
    
pdensOut = arcpy.sa.PointDensity(os.path.join(GIS_FOLDER, 'Direct_infiltration\\OSDS_units_pts.shp'),"NONE", cel_size, NbrCircle(1, "CELL"), "SQUARE_METERS" )
pdensOut.save(os.path.join(workspace, 'temp_dens'))   # calculate the OSDS density per m2

arcpy.Times_3d(os.path.join(workspace, 'temp_dens'), cel_size**2, os.path.join(workspace, 'OSDS_dens'))  # make the OSDS density in OSDS units, meaning number of units per cell

infiltration_in_inches_per_day_per_OSDS = ((1.454203/cel_size**2)*39.3701)  +  (((1.454203/cel_size**2)*39.3701)*DIFF_from_scenario_amount)

arcpy.Times_3d(os.path.join(workspace, 'OSDS_dens'), infiltration_in_inches_per_day_per_OSDS, os.path.join(workspace, 'OSDS_inlf_in'))  # calculate amount of water infiltrated in each cell in inches

arcpy.RasterToASCII_conversion(os.path.join(workspace, 'OSDS_inlf_in'), os.path.join(workspace, "OSDS_inlf_in.asc"))

arcpy.Delete_management(os.path.join(workspace, 'temp_dens'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_dens'))   # clean up workspace

#### add all the direct infiltration into one raster because apparently the control file wont take multiples???
arcpy.Plus_3d(os.path.join(workspace,  "A_WL_Rast.asc"), os.path.join(workspace, "OSDS_inlf_in.asc"), os.path.join(workspace, "WLOSDrast"))
arcpy.RasterToASCII_conversion(os.path.join(workspace, "WLOSDrast"), os.path.join(workspace, "Total_inlf_in.asc"))

# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "Total_inlf_in.asc"))

### Scen 2 water lines  and OSDS location infiltration

In [11]:
workspace = os.path.join('..', 'input/Direct_infiltration_Scen_2')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
########   Main change Parameter HERE   ############
DIFF_from_scenario_amount = -.33

    
# create appropriately sized buffers of 1/2 the cell size
arcpy.Buffer_analysis(os.path.join(GIS_FOLDER, 'Direct_infiltration\\Transmission_water_mains.shp'), os.path.join(workspace, 'Water_lines_Buffer.shp'), "{} meters".format(cel_size/2), "FULL", "ROUND", "ALL")
# calculate area of water line polygons in m2
arcpy.AddField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "DOUBLE")    # add Active cell unit field
arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "AREA")    # calculate the area of the buffer zone
Area_list= [row[0] for row in arcpy.da.SearchCursor(os.path.join(workspace, 'Water_lines_Buffer.shp'), "POLY_AREA")]  # make a list of the area number
arcpy.CalculateField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "!Id! + 30056.1755/Area_list[0]*39.3701", "PYTHON3") # calculate the appropriate amount of infitration in inches to equal 7.94 MGD as spread over the whole water line influence area
    
# Project dataset into WGS84
arcpy.Project_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'), sr_project) 
arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'),  os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp')], os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))

arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'), "Infl_inch", os.path.join(workspace, "A_WL_Rast"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "A_WL_Rast"), os.path.join(workspace, "A_WL_Rast.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'A_WL_Rast'))



### Rasterize OSDS location infiltration
# FROM Shuler et al 2017   OSDS flow = 1.454203 m3/unit/Day
    
arcpy.env.extent = os.path.join('..', 'input/General/Grid_rast1')  # set processing extent to our desired model boundary
    
pdensOut = arcpy.sa.PointDensity(os.path.join(GIS_FOLDER, 'Direct_infiltration\\OSDS_units_pts.shp'),"NONE", cel_size, NbrCircle(1, "CELL"), "SQUARE_METERS" )
pdensOut.save(os.path.join(workspace, 'temp_dens'))   # calculate the OSDS density per m2

arcpy.Times_3d(os.path.join(workspace, 'temp_dens'), cel_size**2, os.path.join(workspace, 'OSDS_dens'))  # make the OSDS density in OSDS units, meaning number of units per cell

infiltration_in_inches_per_day_per_OSDS = ((1.454203/cel_size**2)*39.3701)  +  (((1.454203/cel_size**2)*39.3701)*DIFF_from_scenario_amount)

arcpy.Times_3d(os.path.join(workspace, 'OSDS_dens'), infiltration_in_inches_per_day_per_OSDS, os.path.join(workspace, 'OSDS_inlf_in'))  # calculate amount of water infiltrated in each cell in inches

arcpy.RasterToASCII_conversion(os.path.join(workspace, 'OSDS_inlf_in'), os.path.join(workspace, "OSDS_inlf_in.asc"))

arcpy.Delete_management(os.path.join(workspace, 'temp_dens'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_dens'))   # clean up workspace

#### add all the direct infiltration into one raster because apparently the control file wont take multiples???
arcpy.Plus_3d(os.path.join(workspace,  "A_WL_Rast.asc"), os.path.join(workspace, "OSDS_inlf_in.asc"), os.path.join(workspace, "WLOSDrast"))
arcpy.RasterToASCII_conversion(os.path.join(workspace, "WLOSDrast"), os.path.join(workspace, "Total_inlf_in.asc"))

# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "Total_inlf_in.asc"))

### Scen 3 water lines  and OSDS location infiltration

In [12]:
workspace = os.path.join('..', 'input/Direct_infiltration_Scen_3')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
########   Main change Parameter HERE   ############
DIFF_from_scenario_amount = 0.64

    
# create appropriately sized buffers of 1/2 the cell size
arcpy.Buffer_analysis(os.path.join(GIS_FOLDER, 'Direct_infiltration\\Transmission_water_mains.shp'), os.path.join(workspace, 'Water_lines_Buffer.shp'), "{} meters".format(cel_size/2), "FULL", "ROUND", "ALL")
# calculate area of water line polygons in m2
arcpy.AddField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "DOUBLE")    # add Active cell unit field
arcpy.AddGeometryAttributes_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "AREA")    # calculate the area of the buffer zone
Area_list= [row[0] for row in arcpy.da.SearchCursor(os.path.join(workspace, 'Water_lines_Buffer.shp'), "POLY_AREA")]  # make a list of the area number
arcpy.CalculateField_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), "Infl_inch", "!Id! + 30056.1755/Area_list[0]*39.3701", "PYTHON3") # calculate the appropriate amount of infitration in inches to equal 7.94 MGD as spread over the whole water line influence area
    
# Project dataset into WGS84
arcpy.Project_management(os.path.join(workspace, 'Water_lines_Buffer.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'), sr_project) 
arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'),  os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'), os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp')], os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))

arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'), "Infl_inch", os.path.join(workspace, "A_WL_Rast"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "A_WL_Rast"), os.path.join(workspace, "A_WL_Rast.asc"))

# clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_projected.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_bound.shp'))
arcpy.Delete_management(os.path.join(workspace, 'Water_lines_Buffer_shp_ready.shp'))
arcpy.Delete_management(os.path.join(workspace, 'A_WL_Rast'))



### Rasterize OSDS location infiltration
# FROM Shuler et al 2017   OSDS flow = 1.454203 m3/unit/Day
    
arcpy.env.extent = os.path.join('..', 'input/General/Grid_rast1')  # set processing extent to our desired model boundary
    
pdensOut = arcpy.sa.PointDensity(os.path.join(GIS_FOLDER, 'Direct_infiltration\\OSDS_units_pts.shp'),"NONE", cel_size, NbrCircle(1, "CELL"), "SQUARE_METERS" )
pdensOut.save(os.path.join(workspace, 'temp_dens'))   # calculate the OSDS density per m2

arcpy.Times_3d(os.path.join(workspace, 'temp_dens'), cel_size**2, os.path.join(workspace, 'OSDS_dens'))  # make the OSDS density in OSDS units, meaning number of units per cell

infiltration_in_inches_per_day_per_OSDS = ((1.454203/cel_size**2)*39.3701)  +  (((1.454203/cel_size**2)*39.3701)*DIFF_from_scenario_amount)

arcpy.Times_3d(os.path.join(workspace, 'OSDS_dens'), infiltration_in_inches_per_day_per_OSDS, os.path.join(workspace, 'OSDS_inlf_in'))  # calculate amount of water infiltrated in each cell in inches

arcpy.RasterToASCII_conversion(os.path.join(workspace, 'OSDS_inlf_in'), os.path.join(workspace, "OSDS_inlf_in.asc"))

arcpy.Delete_management(os.path.join(workspace, 'temp_dens'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_inlf_in'))   # clean up workspace
arcpy.Delete_management(os.path.join(workspace, 'OSDS_dens'))   # clean up workspace

#### add all the direct infiltration into one raster because apparently the control file wont take multiples???
arcpy.Plus_3d(os.path.join(workspace,  "A_WL_Rast.asc"), os.path.join(workspace, "OSDS_inlf_in.asc"), os.path.join(workspace, "WLOSDrast"))
arcpy.RasterToASCII_conversion(os.path.join(workspace, "WLOSDrast"), os.path.join(workspace, "Total_inlf_in.asc"))

# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "Total_inlf_in.asc"))

### Create Flow direction raster

In [13]:
### Create Flow direction raster
####resampling the 10 m DEM to whatever cell size we is using 
workspace = os.path.join('..', 'input/DEM_Process')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
# this resamples the 10 , dem to whatever cell size we are working in
arcpy.ProjectRaster_management(os.path.join( GIS_FOLDER, 'DEM', '10M_DEM.tif'), os.path.join(workspace, 'DEM_projected.tif'), sr_project, cell_size = cel_size )
arcpy.Clip_management(os.path.join(workspace, 'DEM_projected.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'DEM_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
arcpy.Plus_3d(os.path.join(workspace, 'DEM_clip.tif'), Grid_rast0,  os.path.join(workspace, 'DEM_ready.tif'))

outFlowDirection = FlowDirection(os.path.join(workspace, 'DEM_ready.tif'), "NORMAL")
outFlowDirection.save(os.path.join(workspace,'Flow_direction.tif'))
arcpy.RasterToASCII_conversion(os.path.join(workspace,'Flow_direction.tif'), os.path.join(workspace,'Flow_direction.asc'))

arcpy.Delete_management(os.path.join(workspace, 'DEM_projected.tif'))
arcpy.Delete_management(os.path.join(workspace, 'DEM_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'DEM_ready.tif'))
arcpy.Delete_management(os.path.join(workspace, 'Flow_direction.tif'))

# Check on existance of no data values
Check_no_data_asc(os.path.join(workspace, "Flow_direction.asc"))

### ORIGINAL mean annual wet canopy evaporation layer

In [14]:
### Create mean annual wet canopy evaporation layer from wind and rainfall data
workspace = os.path.join('..', 'input/Evaporation')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
#files to use    
anual_rain = os.path.join(GIS_FOLDER, 'Gridded_rain', "year", "An_pcip_in.tif")
anual_wind =  os.path.join(GIS_FOLDER, 'Wind_map', 'wndsp_30m.tif')

# divide them to get w
outDivide = Divide(anual_wind, anual_rain)                     
outDivide.save(os.path.join(workspace, "w_WndOvrRain.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches

# now apply equation 18  V = 2.677 × (w) – 0.014, from "Spatially Distributed Groundwater Recharge for 2010 Land Cover Estimated Using a Water-Budget Model for the Island of O’ahu, Hawai’i"
outTimes = Times(os.path.join(workspace, "w_WndOvrRain.tif"), 2.677)                     
outTimes.save(os.path.join(workspace, "w_Wndtmp.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches
arcpy.Plus_3d(os.path.join(workspace, "w_Wndtmp.tif"),  -0.014, os.path.join(workspace, "V_evp2pcip.tif"))

# synchronize geometry with the model
arcpy.ProjectRaster_management(os.path.join(workspace, "V_evp2pcip.tif"), os.path.join(workspace, 'V_evp_projected.tif'), sr_project, cell_size = cel_size )
arcpy.Clip_management(os.path.join(workspace, 'V_evp_projected.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'V_evp_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
arcpy.Plus_3d(os.path.join(workspace, 'V_evp_clip.tif'), Grid_rast0,  os.path.join(workspace, 'V_evp_ready.tif'))
# print .asc file
arcpy.RasterToASCII_conversion(os.path.join(workspace,'V_evp_ready.tif'), os.path.join(workspace,'V_evp2pcp.asc'))   

#clean
arcpy.Delete_management(os.path.join(workspace, 'w_WndOvrRain.tif'))
arcpy.Delete_management(os.path.join(workspace, 'w_Wndtmp.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp2pcip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_projected.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_ready.tif'))


<Result 'true'>

### RCP_PRESENT mean annual wet canopy evaporation layer

In [15]:
### Create mean annual wet canopy evaporation layer from wind and rainfall data
workspace = os.path.join('..', 'input/Evaporation_RCP_Present')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
    
########   Main change Parameter HERE   ############
#files to use    
anual_rain = os.path.join(GIS_FOLDER, 'Climate_scenarios', "ProcessedData", "yearly_rainfall_present.asc")
anual_wind =  os.path.join(GIS_FOLDER, 'Wind_map', 'wndsp_30m.tif')

# divide them to get w
outDivide = Divide(anual_wind, anual_rain)                     
outDivide.save(os.path.join(workspace, "w_WndOvrRain.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches

# now apply equation 18  V = 2.677 × (w) – 0.014, from "Spatially Distributed Groundwater Recharge for 2010 Land Cover Estimated Using a Water-Budget Model for the Island of O’ahu, Hawai’i"
outTimes = Times(os.path.join(workspace, "w_WndOvrRain.tif"), 2.677)                     
outTimes.save(os.path.join(workspace, "w_Wndtmp.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches
arcpy.Plus_3d(os.path.join(workspace, "w_Wndtmp.tif"),  -0.014, os.path.join(workspace, "V_evp2pcip.tif"))

# synchronize geometry with the model
arcpy.ProjectRaster_management(os.path.join(workspace, "V_evp2pcip.tif"), os.path.join(workspace, 'V_evp_projected.tif'), sr_project, cell_size = cel_size )
arcpy.Clip_management(os.path.join(workspace, 'V_evp_projected.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'V_evp_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
arcpy.Plus_3d(os.path.join(workspace, 'V_evp_clip.tif'), Grid_rast0,  os.path.join(workspace, 'V_evp_ready.tif'))
# print .asc file
arcpy.RasterToASCII_conversion(os.path.join(workspace,'V_evp_ready.tif'), os.path.join(workspace,'V_evp2pcp.asc'))   

#clean
arcpy.Delete_management(os.path.join(workspace, 'w_WndOvrRain.tif'))
arcpy.Delete_management(os.path.join(workspace, 'w_Wndtmp.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp2pcip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_projected.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_ready.tif'))    

<Result 'true'>

### RCP4.5 mean annual wet canopy evaporation layer

In [16]:
### Create mean annual wet canopy evaporation layer from wind and rainfall data
workspace = os.path.join('..', 'input/Evaporation_RCP4p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    

 ########   Main change Parameter HERE   ############
#files to use    
anual_rain = os.path.join(GIS_FOLDER, 'Climate_scenarios', "ProcessedData", "yearly_rainfall_rcp45.asc")
anual_wind =  os.path.join(GIS_FOLDER, 'Wind_map', 'wndsp_30m.tif')

# divide them to get w
outDivide = Divide(anual_wind, anual_rain)                     
outDivide.save(os.path.join(workspace, "w_WndOvrRain.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches

# now apply equation 18  V = 2.677 × (w) – 0.014, from "Spatially Distributed Groundwater Recharge for 2010 Land Cover Estimated Using a Water-Budget Model for the Island of O’ahu, Hawai’i"
outTimes = Times(os.path.join(workspace, "w_WndOvrRain.tif"), 2.677)                     
outTimes.save(os.path.join(workspace, "w_Wndtmp.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches
arcpy.Plus_3d(os.path.join(workspace, "w_Wndtmp.tif"),  -0.014, os.path.join(workspace, "V_evp2pcip.tif"))

# synchronize geometry with the model
arcpy.ProjectRaster_management(os.path.join(workspace, "V_evp2pcip.tif"), os.path.join(workspace, 'V_evp_projected.tif'), sr_project, cell_size = cel_size )
arcpy.Clip_management(os.path.join(workspace, 'V_evp_projected.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'V_evp_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
arcpy.Plus_3d(os.path.join(workspace, 'V_evp_clip.tif'), Grid_rast0,  os.path.join(workspace, 'V_evp_ready.tif'))
# print .asc file
arcpy.RasterToASCII_conversion(os.path.join(workspace,'V_evp_ready.tif'), os.path.join(workspace,'V_evp2pcp.asc'))   

#clean
arcpy.Delete_management(os.path.join(workspace, 'w_WndOvrRain.tif'))
arcpy.Delete_management(os.path.join(workspace, 'w_Wndtmp.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp2pcip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_projected.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_ready.tif'))     



<Result 'true'>

### RCP8.5 mean annual wet canopy evaporation layer

In [17]:
### Create mean annual wet canopy evaporation layer from wind and rainfall data
workspace = os.path.join('..', 'input/Evaporation_RCP8p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    

 ########   Main change Parameter HERE   ############
#files to use    
anual_rain = os.path.join(GIS_FOLDER, 'Climate_scenarios', "ProcessedData", "yearly_rainfall_rcp85.asc")
anual_wind =  os.path.join(GIS_FOLDER, 'Wind_map', 'wndsp_30m.tif')

# divide them to get w
outDivide = Divide(anual_wind, anual_rain)                     
outDivide.save(os.path.join(workspace, "w_WndOvrRain.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches

# now apply equation 18  V = 2.677 × (w) – 0.014, from "Spatially Distributed Groundwater Recharge for 2010 Land Cover Estimated Using a Water-Budget Model for the Island of O’ahu, Hawai’i"
outTimes = Times(os.path.join(workspace, "w_WndOvrRain.tif"), 2.677)                     
outTimes.save(os.path.join(workspace, "w_Wndtmp.tif")) # create variable w = quotient of mean annual wind speed, in meters per second, and mean annual rainfall, in inches
arcpy.Plus_3d(os.path.join(workspace, "w_Wndtmp.tif"),  -0.014, os.path.join(workspace, "V_evp2pcip.tif"))

# synchronize geometry with the model
arcpy.ProjectRaster_management(os.path.join(workspace, "V_evp2pcip.tif"), os.path.join(workspace, 'V_evp_projected.tif'), sr_project, cell_size = cel_size )
arcpy.Clip_management(os.path.join(workspace, 'V_evp_projected.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'V_evp_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
arcpy.Plus_3d(os.path.join(workspace, 'V_evp_clip.tif'), Grid_rast0,  os.path.join(workspace, 'V_evp_ready.tif'))
# print .asc file
arcpy.RasterToASCII_conversion(os.path.join(workspace,'V_evp_ready.tif'), os.path.join(workspace,'V_evp2pcp.asc'))   

#clean
arcpy.Delete_management(os.path.join(workspace, 'w_WndOvrRain.tif'))
arcpy.Delete_management(os.path.join(workspace, 'w_Wndtmp.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp2pcip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_projected.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'V_evp_ready.tif'))    

<Result 'true'>

### ORIGINAL rainfall grids

In [18]:
### format raingall grids
workspace = os.path.join('..', 'input/Gridded_rain')
if not os.path.exists(workspace):
    os.makedirs(workspace)

mo_list = ["01", "02", "03","04", "05", "06","07", "08", "09","10", "11", "12"]

for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Gridded_rain', "PRISM_ppt_tutuila_30yr_normal_80mM1_{}_asc.asc".format(i))
    outpy = os.path.join(workspace, 'PRISM_ppt_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, "BILINEAR", cel_size, "NAD_1983_To_WGS_1984_1", "#", "#") 
    outpy2 = Times(outpy, 0.000393701)   # needed to convert 100* mm to inches
    arcpy.Clip_management(outpy2, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    
    
    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'PRISM_ppt_tutuila_30yr_normal_{}.asc'.format(i)))
    

    
arcpy.Delete_management(os.path.join(workspace, 'PRISM_ppt_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### RCP_PRESENT raingall grids  

In [19]:
### format raingall grids
workspace = os.path.join('..', 'input/Gridded_rain_RCP_Present')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
    
mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]

########   Main change Parameter HERE   ############
nam = "present"

  
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_rainfall_{}.asc".format(i, nam))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'CChanged_ppt_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size) 
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_ppt_{}_month_{}.asc'.format(nam, i)))
    
arcpy.Delete_management(os.path.join(workspace, 'CChanged_ppt_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### RCP4.5  raingall grids

In [20]:
### format raingall grids
workspace = os.path.join('..', 'input/Gridded_rain_RCP_RCP4p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
       
mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]

########   Main change Parameter HERE   ############
nam = "rcp45"

  
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_rainfall_{}.asc".format(i, nam))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'CChanged_ppt_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size) 
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_ppt_{}_month_{}.asc'.format(nam, i)))
    
arcpy.Delete_management(os.path.join(workspace, 'CChanged_ppt_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### RCP8.5 raingall grids

In [21]:
### format raingall grids
workspace = os.path.join('..', 'input/Gridded_rain_RCP_RCP8p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    

mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]

########   Main change Parameter HERE   ############
nam = "rcp85"

  
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_rainfall_{}.asc".format(i, nam))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'CChanged_ppt_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size) 
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_ppt_{}_month_{}.asc'.format(nam, i)))
    
arcpy.Delete_management(os.path.join(workspace, 'CChanged_ppt_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### ORIGINAL Air Temp grids

In [22]:
### format Air Temp grids
workspace = os.path.join('..', 'input/Gridded_temps')
if not os.path.exists(workspace):
    os.makedirs(workspace)

mo_list = ["01", "02", "03","04", "05", "06","07", "08", "09","10", "11", "12"]

#max temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Gridded_temps/Temp_max/Monthly', "PRISM_tmax_tutuila_30yr_normal_80mM1_{}_asc.asc".format(i))
    outpy = os.path.join(workspace,'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, "BILINEAR", cel_size, "NAD_1983_To_WGS_1984_1", "#", "#") 
    outpy2 = Times(outpy, 0.01)   # needed to convert to deg celsius
    arcpy.Clip_management(outpy2, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    
    
    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'PRISM_tmax_tutuila_30yr_normal_{}.asc'.format(i)))
    
# min temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Gridded_temps/Temp_min/Monthly', "PRISM_tmin_tutuila_30yr_normal_80mM1_{}_asc.asc".format(i))
    outpy = os.path.join(workspace, 'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, "BILINEAR", cel_size, "NAD_1983_To_WGS_1984_1", "#", "#") 
    outpy2 = Times(outpy, 0.01)   # needed to convert to deg celsius
    arcpy.Clip_management(outpy2, "515000 8429000 550000 8409000", os.path.join(workspace,'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    
    
    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'PRISM_tmin_tutuila_30yr_normal_{}.asc'.format(i)))    
    
arcpy.Delete_management(os.path.join(workspace, 'PRISM_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### RCP_PRESENT Air Temp  grids     

In [23]:
### format Air Temp grids
workspace = os.path.join('..', 'input/Gridded_temps_RCP_Present')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]
# name_list = ["present", "rcp45", "rcp85"]

########   Main change Parameter HERE   ############
n = "present"


# for n in name_list:
#max temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_maxTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace,'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_maxT_{}_month_{}.asc'.format(n, i)))

# min temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_minTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace,'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_minT_{}_month_{}.asc'.format(n, i)))    

arcpy.Delete_management(os.path.join(workspace, 'PRISM_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))

<Result 'true'>

### RCP4.5 Air Temp  grids 

In [24]:
### format Air Temp grids
workspace = os.path.join('..', 'input/Gridded_temps_RCP_RCP4p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    

mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]
# name_list = ["present", "rcp45", "rcp85"]

########   Main change Parameter HERE   ############
n = "rcp45"


# for n in name_list:
#max temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_maxTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace,'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_maxT_{}_month_{}.asc'.format(n, i)))

# min temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_minTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace,'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_minT_{}_month_{}.asc'.format(n, i)))    

arcpy.Delete_management(os.path.join(workspace, 'PRISM_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))


<Result 'true'>

### RCP8.5 Air Temp  grids 

In [25]:
### format Air Temp grids
workspace = os.path.join('..', 'input/Gridded_temps_RCP_RCP8p5')
if not os.path.exists(workspace):
    os.makedirs(workspace)
    

mo_list = ["1", "2", "3","4", "5", "6","7", "8", "9","10", "11", "12"]
# name_list = ["present", "rcp45", "rcp85"]

########   Main change Parameter HERE   ############
n = "rcp85"


# for n in name_list:
#max temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_maxTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace,'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_maxT_{}_month_{}.asc'.format(n, i)))

# min temps
for i in mo_list:
    inpt =  os.path.join(GIS_FOLDER, 'Climate_scenarios', 'ProcessedData', "month_{}_minTemp_{}.asc".format(i, n))
    arcpy.DefineProjection_management(inpt, sr_project)
    outpy = os.path.join(workspace, 'PRISM_clip.tif')

    arcpy.ProjectRaster_management(inpt, outpy, sr_project, cell_size = cel_size)
    arcpy.Clip_management(outpy, "515000 8429000 550000 8409000", os.path.join(workspace,'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))    

    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, 'CChanged_minT_{}_month_{}.asc'.format(n, i)))    

arcpy.Delete_management(os.path.join(workspace, 'PRISM_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))
arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif')) 

<Result 'true'>

### Create Monthly Evapotranspiration grids from the original Izuka 2005 paper maps

In [26]:
### Create Monthly Evapotranspiration grids from the original Izuka 2005 paper maps
#Spline method 
workspace = os.path.join('..', 'input//ET_Process')
if not os.path.exists(workspace):
    os.makedirs(workspace)

files = ['Jan_ET.shp', 'Feb_ET.shp', 'March_ET.shp', 'April_ET.shp', 'May_ET.shp', 'June_ET.shp', 'July_ET.shp', 'Aug_ET.shp', 'Sep_ET.shp', 'Oct_ET.shp', 'Nov-Dec_ET.shp', 'Nov-Dec_ET.shp']
mos = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec",]

for idx, i in enumerate(files):
    key_nam = mos[idx]
    outSPL = Spline(os.path.join(GIS_FOLDER, 'ET', i), "ET_inches", cel_size, "REGULARIZED")   # spline the points
    outSPL.save(os.path.join(workspace, 'ET.tif'))
    
    outExtractByMask = ExtractByMask(outSPL, os.path.join(GIS_FOLDER, 'Precip_bound_poly.shp'))                  # clip the raster
    outExtractByMask.save(os.path.join(workspace, 'ET_clipped.tif'))
    
# already projected   arcpy.ProjectRaster_management(os.path.join(workspace, 'junk' 'ET_clipped.tif'), os.path.join(workspace, 'junk', 'ET_proj.tif'), sr_project, cell_size = cel_size)
    arcpy.Clip_management(os.path.join(workspace, 'ET_clipped.tif'), "515000 8429000 550000 8409000", os.path.join(workspace, 'file_clip.tif'), Grid_shp, -9999, 'ClippingGeometry', 'MAINTAIN_EXTENT')   # make sure raster extent is good
    arcpy.Plus_3d(os.path.join(workspace, 'file_clip.tif'), Grid_rast0,  os.path.join(workspace, 'file_plus.tif'))
    
    arcpy.RasterToASCII_conversion(os.path.join(workspace, 'file_plus.tif'), os.path.join(workspace, key_nam+'_ET_clipped.asc'))    

    arcpy.Delete_management(os.path.join(workspace, 'ET.tif'))                                                     # delete extranious file
    arcpy.Delete_management(os.path.join(workspace, 'ET_clipped.tif'))    
    arcpy.Delete_management(os.path.join(workspace, 'file_clip.tif'))    
    arcpy.Delete_management(os.path.join(workspace, 'file_plus.tif'))  

### Silly rainfall adjustment grids,

In [27]:
### Silly rainfall adjustment grids,  not actual data, just a filler grid
workspace = os.path.join('..', 'input//RF_adj_grids')
if not os.path.exists(workspace):
    os.makedirs(workspace)

Field_to_rasterize = "one"

mos = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec",]

for i in mos:
    Gen_name = "RFadj_{}".format(i)

    arcpy.PolygonToRaster_conversion(Grid_shp, Field_to_rasterize, os.path.join(workspace, '{}'.format(Gen_name)), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
    arcpy.RasterToASCII_conversion( os.path.join(workspace, '{}'.format(Gen_name)), os.path.join(workspace, "{}.asc".format(Gen_name)))
